In [1]:
# This examples shows who to download files from the ONC server

import numpy as np

import strawb
import strawb.sensors.camera as camera

## Download the files from the ONC server

In [2]:
onc_downloader = strawb.ONCDownloader(showInfo=False)

filters = {'deviceCode': 'TUMMINISPECTROMETER001',
           'dateFrom': '2021-05-01T19:00:00.000Z',
           'dateTo': '2021-05-01T21:59:59.000Z',
           'extension': 'hdf5'}

# download in foreground
onc_downloader.download_file(filters=filters, allPages=True)

Download in directory: /Users/ruohanli/Documents/Programs/strawb_package/raw_module_data
Obtained a list of 5 files to download.
   Skipping "TUMMINISPECTROMETER001_20210501T000000.000Z-SDAQ-MINISPEC.hdf5": File already exists.
   Skipping "TUMMINISPECTROMETER001_20210501T000000.000Z-SDAQ-MODULE.hdf5": File already exists.
   Skipping "TUMMINISPECTROMETER001_20210501T190000.000Z-SDAQ-CAMERA.hdf5": File already exists.
   Skipping "TUMMINISPECTROMETER001_20210501T200000.000Z-SDAQ-CAMERA.hdf5": File already exists.
   Skipping "TUMMINISPECTROMETER001_20210501T210000.000Z-SDAQ-CAMERA.hdf5": File already exists.
0 files (0 Bytes) downloaded
Total Download Time: 0.000 seconds


# Load a single File

In [5]:
file_name = 'TUMMINISPECTROMETER001_20210501T190000.000Z-SDAQ-CAMERA.hdf5'
cam_run = camera.FileHandler(file_name)

In [6]:
# print some parameters
print(f'Module: {cam_run.module}')
print(f'Number of Frames: {cam_run.exposure_time.shape[0]}')
print(f'Date: {np.min(cam_run.time)} - {np.max(cam_run.time)}')
print(f'Exposure Times [s]: {np.unique(cam_run.exposure_time)}')

Module: MINISPECTROMETER001
Number of Frames: 36
Date: 2021-05-01T18:59:33.334768 - 2021-05-01T19:58:09.306084
Exposure Times [s]: [63.5]


In [7]:
# Rhe raw pixel values are NOT loaded by default to the module to save RAM.
# They can be accessed directly from the file with the index, index = None (default) to loads all images
cam_run.load_raw(index=[1,3,4])

array([[[    0,     0,     0, ..., 20384, 31424, 18784],
        [    0,  7904,     0, ..., 25504, 30144, 19264],
        [ 1984,  5824,  5344, ..., 28224, 32864, 19104],
        ...,
        [53824, 65504, 65504, ..., 65504, 65504, 65504],
        [65504, 65504, 65504, ..., 65504, 65504, 65504],
        [65504, 65504, 65504, ..., 65504, 65504, 65504]],

       [[    0,     0,     0, ..., 25920, 32480, 37120],
        [  640,     0,  2400, ..., 27520, 28320, 29280],
        [ 1440,   480,     0, ..., 36160, 22240, 34560],
        ...,
        [65504, 65504, 65504, ..., 65504, 65504, 65504],
        [65504, 65504, 65504, ..., 65504, 65504, 65504],
        [63840, 65504, 64160, ..., 65504, 65504, 65504]],

       [[    0,     0,     0, ..., 36592, 41392, 14032],
        [ 5392,  4752,  4112, ..., 28592, 31472, 23312],
        [ 1552,     0,   112, ..., 21392, 12432, 28912],
        ...,
        [65504, 60432, 60912, ..., 65504, 65504, 65504],
        [65504, 65504, 65504, ..., 65504, 655

### Store the images with Lucifer enabled

In [8]:
cam_run.image2png_lucifer()

save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/TORCH_0_15/2021_05_01_19_24_37.png
save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/TORCH_1_15/2021_05_01_19_26_21.png
save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/TORCH_2_15/2021_05_01_19_28_04.png
save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/TORCH_3_15/2021_05_01_19_29_45.png
save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/TORCH_4_15/2021_05_01_19_31_27.png
save: /Users/ruohanli/Documents/Programs/strawb_package/processed_data/MINISPECTROMETER001/FLASH_0_15_7/2021_05_01_19_19_34.png


In [11]:
test_str = "2021_05_01_19_19_34.png"
test_str[11:13]

'19'

### Store the 'brightest' images

In [23]:
mode_list, mask_list = cam_run.get_lucifer_mask()  # returns a mask for each lucifer setting
mask_lucifer = np.any(mask_list, axis=0)  # coomopress the masks to one mask

# mask over a threshold + mask invalid frames + mask no lucifer enabled
mask = (cam_run.integrated_minus_dark > 1e3) & cam_run.invalid_mask & ~mask_lucifer

index = np.argsort(cam_run.integrated_minus_dark)  # sort by charge [min,...,max]
index = index[mask[index]]  # remove invalid items  & cam_module.invalid_mask
index = index[::-1]  # revers the order
cam_run.image2png(index=index, f_name_formatter='{i}_{datetime}.png')  # save the images

save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/0_2021_05_01_19_26_56.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/1_2021_05_01_19_21_57.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/2_2021_05_01_19_35_14.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/3_2021_05_01_19_40_13.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/4_2021_05_01_19_36_53.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/5_2021_05_01_19_46_51.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/6_2021_05_01_19_51_50.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/7_2021_05_01_19_25_16.png
save: /Users/kilian/PycharmProjects/strawb_package/processed_data/PMTSPECTROMETER002/8_2021_05_01_19_10_20.png
s